In [450]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [451]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
#null values in train data age : 177, cabin : 687, embarked : 2
#null values in test data age: 86 , cabin : 327, fare : 1

# Analysing

In [452]:
train_data.shape, test_data.shape

((891, 12), (418, 11))

In [453]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [454]:
train_data['Survived'].value_counts() #342 survived

0    549
1    342
Name: Survived, dtype: int64

In [455]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [456]:
P_S_Embark_S = 217/(427+217) #probability survival in embark s
P_S_Embark_C = 93/(93+75)
P_S_Embark_Q = 30/(30+47)
print("Probability of survival in Embark_S", P_S_Embark_S)
print("Probability of survival in Embark_C", P_S_Embark_C)
print("Probability of survival in Embark_Q", P_S_Embark_Q)
#so c embark people survived more

Probability of survival in Embark_S 0.33695652173913043
Probability of survival in Embark_C 0.5535714285714286
Probability of survival in Embark_Q 0.38961038961038963


In [457]:
Total_died = train_data['Sex'][train_data['Survived']==0].value_counts()
Total_Survived = train_data['Sex'][train_data['Survived']==1].value_counts()
Total_died, Total_Survived

(male      468
 female     81
 Name: Sex, dtype: int64,
 female    233
 male      109
 Name: Sex, dtype: int64)

In [458]:
print("Probability of men survival :", 109/(109+468))
print("Probability of female survival", 233/(233+81))

Probability of men survival : 0.18890814558058924
Probability of female survival 0.7420382165605095


# Data Preprocessing

In [459]:
train_data.drop(['PassengerId'], axis = 1, inplace = True)

In [460]:
train_data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Missing data

In [461]:
train_data.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [462]:
datasets = [train_data, test_data]
avg_age = train_data['Age'].sum()/714
for i in datasets:
    i['Age'].fillna(avg_age, inplace = True)
train_data.isnull().sum()
#Missing age data cleared

Survived      0
Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [463]:
train_data['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [464]:
train_data['Embarked'].fillna('S', inplace = True)
train_data.isnull().sum()
#Missing embarked data cleared

Survived      0
Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      0
dtype: int64

In [465]:
total_data = [train_data, test_data]
for i in total_data:
    

    i.drop(columns = ['Cabin'], inplace = True)
    i.Sex[i.Sex == 'male'] =1
    i.Sex[i.Sex == 'female'] =0
    i.Embarked[i.Embarked == 'S'] =0
    i.Embarked[i.Embarked == 'C'] =1
    i.Embarked[i.Embarked == 'Q'] =2

In [466]:
train_data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",0,38.000000,1,0,PC 17599,71.2833,1
2,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,0
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,0
4,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",1,27.000000,0,0,211536,13.0000,0
887,1,1,"Graham, Miss. Margaret Edith",0,19.000000,0,0,112053,30.0000,0
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,29.699118,1,2,W./C. 6607,23.4500,0
889,1,1,"Behr, Mr. Karl Howell",1,26.000000,0,0,111369,30.0000,1


In [467]:
data = [train_data, test_data]
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    # extract titles
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)
train_data = train_data.drop(['Name'], axis=1)
test_data = test_data.drop(['Name'], axis=1)

In [469]:
x = train_data.drop(columns = ['Survived','Ticket'])
y = train_data['Survived']

x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
 7   Title     891 non-null    int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [470]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test, y_train, y_test = tts(x,y,test_size = 0.2)

x_train.shape, y_train.shape

((712, 8), (712,))

In [473]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [474]:
predicted_values = neigh.predict(x_test)
predicted_values

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 1], dtype=int64)

In [475]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,predicted_values)
score

0.7150837988826816

In [476]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(class_weight = 'balanced')

In [477]:
classifier.fit(x_train,y_train)
pred_values = classifier.predict(x_test)
pred_values

array([0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 1], dtype=int64)

In [478]:
x_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
864,2,1,24.000000,0,0,13.0000,0,1
865,2,0,42.000000,0,0,13.0000,0,3
372,3,1,19.000000,0,0,8.0500,0,1
775,3,1,18.000000,0,0,7.7500,0,1
203,3,1,45.500000,0,0,7.2250,1,1
...,...,...,...,...,...,...,...,...
630,1,1,80.000000,0,0,30.0000,0,1
318,1,0,31.000000,0,2,164.8667,0,2
757,2,1,18.000000,0,0,11.5000,0,1
773,3,1,29.699118,0,0,7.2250,1,1


In [479]:
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
115,3,1,21.000000,0,0,7.9250,0,1
670,2,0,40.000000,1,1,39.0000,0,3
743,3,1,24.000000,1,0,16.1000,0,1
165,3,1,9.000000,0,2,20.5250,0,4
788,3,1,1.000000,1,2,20.5750,0,4
...,...,...,...,...,...,...,...,...
162,3,1,26.000000,0,0,7.7750,0,1
631,3,1,51.000000,0,0,7.0542,0,1
810,3,1,26.000000,0,0,7.8875,0,1
633,1,1,29.699118,0,0,0.0000,0,1


In [480]:
score = accuracy_score(y_test,pred_values)
score

0.7653631284916201

In [ ]:
test_data.drop(columns = ['PassengerId','Cabin','Ticket'], inplace = True)
test_data = pd.get_dummies(test_data, columns = ['Sex', 'Embarked'])

In [ ]:
k = test_data['Fare'].mean()
test_data['Fare'].fillna(k, inplace=True)

In [ ]:
test_data

In [ ]:
pred_values = classifier.predict(test_data)
pred_values

In [ ]:
data_final = pd.DataFrame({'Survived':pred_values})
data_final

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 100)
model.fit(x,y)

pred_values = model.predict(test_data)
pred_values

In [ ]:
data_final = pd.DataFrame({'Survived':pred_values})
data_final

In [ ]:
data_final.to_csv('data_final_22.csv')

In [ ]:
logreg = LogisticRegression()
logreg.fit(x,y)

Y_pred = logreg.predict(test_data)

log_data_final = pd.DataFrame({'Survived':Y_pred})
log_data_final

In [ ]:
log_data_final.to_csv('log_data_final.csv')

In [ ]:
decision_tree = DecisionTreeClassifier() 
decision_tree.fit(x, y)
Y_pred = decision_tree.predict(test_data)  
 

In [ ]:
log_data_final = pd.DataFrame({'Survived':Y_pred})
log_data_final.to_csv('decisiontree_data_final.csv')